In [7]:
from sklearn.decomposition import PCA
from glob import glob
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
import warnings
from astropy.utils.exceptions import AstropyUserWarning
warnings.filterwarnings("ignore", category=AstropyUserWarning)
%matplotlib inline

In [29]:
bin_number = 200
x_min,x_max = 0,4
y_min,y_max = 10, 20

In [30]:
l = []; n = sorted(glob("data/CALIB_LB_???d???JK.rawh5"))

In [56]:
x_width = (x_max-x_min)/bin_number
y_width = (y_max-y_min)/bin_number

In [57]:
pointings_avg = []
H_array   = np.zeros((len(n),bin_number**2))
xed_array = np.zeros((len(n), bin_number))
yed_array = np.zeros((len(n), bin_number))

In [58]:
for ind, val in enumerate(n):
    t = Table.read(val)
    # Filter for photometric errors
    t = t[(t["Jerr"] < 0.2) & (t["Kerr"] < 0.2) & (t["Sharp"] < 1.) & (t["Sharp"] > -1.)]
    df = t.to_pandas()
    pointings_avg.append(np.array(np.nanmedian(df[["L","B"]], axis=0)))
    H, xedges, yedges = np.histogram2d(t["J"]-t["K"], t["K"], bins=bin_number,normed=True,range=[[x_min,x_max],[y_min,y_max]] )
    # Take the middle of histogram edges
    xedges = xedges[:-1]+(x_width/2.)
    yedges = yedges[:-1]+(y_width/2.)
    H_array[ind]   = H.ravel()
    xed_array[ind] = xedges
    yed_array[ind] = yedges

In [59]:
pointings_avg = np.array(pointings_avg)

In [60]:
print (np.mean(df[["L","B"]], axis=0))
pointings_avg = np.array(pointings_avg)

L    350.083935
B      1.214542
dtype: float64


In [ ]:
savehistname = "histogram_edges_mask"
np.savez(savehistname, H_array=H_array, xedges=xedges, yedges=yedges, pointings_avg=pointings_avg)